## 한국수출입은행 - 원화 환율(최근까지) 및 DB 저장
+ 작성: 임경호

### 1. 데이터를 가져올 날짜 설정
+ 저장된 데이터 마지막 일자의 다음날부터 현재일 기준 직전일까지 데이터를 일괄로 가져와서 저장

In [1]:
import requests
import pandas as pd

import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect
import myutil

In [2]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

query = f'SELECT date FROM fx_eximbank ORDER BY date DESC LIMIT 1'
cur.execute(query)

row = cur.fetchone()
last_date = row[0]     # 저장된 데이터의 마지막 일자

conn.close()

# 2023년 1월 1일부터 2023년 6월 30일까지 데이터 일괄 저장
date_from = myutil.get_next_day(last_date)
date_to = myutil.get_previous_day()

### 2. API를 활용한 데이터 수집 (한국수출입은행 Open API)

In [10]:
url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON'
api_key = 'OczbdUJk31QuSPZNx3llq7VFOEmsMuew'
RESULT_SUCCESS = 1

date_range = pd.date_range(start=date_from, end=date_to, freq="D")

df_fx_all = pd.DataFrame()
for dt in date_range:
    date_str = dt.strftime("%Y%m%d")
    params = {
        'authkey': api_key,
        'searchdate': date_str,
        'data': 'AP01'             # 환율
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:     # URL GET '200 정상'
        print("URL GET Error")
        pass
    df_fx = pd.json_normalize(response.json())
    if not df_fx.empty:
        # print(df_fx['result'].unique())
        if RESULT_SUCCESS not in df_fx['result'].unique():         # 성공이 아니라면
            print(f'{date_str}: Result is not successful')
            pass
        df_fx.drop('result', axis=1, inplace=True)
        df_fx.insert(0, 'date', date_str)
        df_fx_all = pd.concat([df_fx_all, df_fx], ignore_index=True)

In [11]:
df_fx_all.shape

(115, 11)

In [12]:
df_fx_all

,date,cur_unit,ttb,tts,deal_bas_r,bkpr,yy_efee_r,ten_dd_efee_r,kftc_bkpr,kftc_deal_bas_r,cur_nm
0,20230713,AED,348.08,355.11,351.6,351,0,0,351,351.6,아랍에미리트 디르함
1,20230713,AUD,867.9,885.43,876.67,876,0,0,876,876.67,호주 달러
2,20230713,BHD,"3,391.3","3,459.81","3,425.56","3,425",0,0,"3,425","3,425.56",바레인 디나르
3,20230713,BND,961.19,980.6,970.9,970,0,0,970,970.9,브루나이 달러
4,20230713,CAD,969.53,989.12,979.33,979,0,0,979,979.33,캐나다 달러
...,...,...,...,...,...,...,...,...,...,...,...
110,20230719,SAR,332.9,339.63,336.27,336,0,0,336,336.27,사우디 리얄
111,20230719,SEK,122.33,124.8,123.57,123,0,0,123,123.57,스웨덴 크로나
112,20230719,SGD,944.34,963.41,953.88,953,0,0,953,953.88,싱가포르 달러
113,20230719,THB,36.66,37.41,37.04,37,0,0,37,37.04,태국 바트


### 3. 데이터 전처리

In [17]:
df_fx_all_save = df_fx_all[['date', 'cur_unit', 'ttb', 'tts', 'deal_bas_r', 'bkpr', 'kftc_bkpr', 'kftc_deal_bas_r']]

In [18]:
# 특정 통화 환율 확인
df_fx_all_save[df_fx_all_save['cur_unit'].str.contains('USD')]

,date,cur_unit,ttb,tts,deal_bas_r,bkpr,kftc_bkpr,kftc_deal_bas_r
22,20230713,USD,"1,278.48","1,304.31","1,291.4","1,291","1,291","1,291.4"
45,20230714,USD,"1,263.43","1,288.96","1,276.2","1,276","1,276","1,276.2"
68,20230717,USD,"1,253.14","1,278.45","1,265.8","1,265","1,265","1,265.8"
91,20230718,USD,"1,253.43","1,278.76","1,266.1","1,266","1,266","1,266.1"
114,20230719,USD,"1,248.98","1,274.21","1,261.6","1,261","1,261","1,261.6"


In [19]:
# 숫자의 소수점(,) 없애기
df_fx_all_save.iloc[:,2:8] = df_fx_all_save.iloc[:,2:8].apply(lambda x: x.str.replace(',', ''), axis = 1)
# 통화코드에서 세 자리 초과하는 내용은 지우고 통화코드 생성: (예) JPY(100) -> JPY로 변경
df_fx_all_save.insert(2, 'cur_code', df_fx_all_save['cur_unit'].str[0:3])
df_fx_all_save = df_fx_all_save.drop('cur_unit', axis=1)

In [20]:
df_fx_all_save

,date,cur_code,ttb,tts,deal_bas_r,bkpr,kftc_bkpr,kftc_deal_bas_r
0,20230713,AED,348.08,355.11,351.6,351,351,351.6
1,20230713,AUD,867.9,885.43,876.67,876,876,876.67
2,20230713,BHD,3391.3,3459.81,3425.56,3425,3425,3425.56
3,20230713,BND,961.19,980.6,970.9,970,970,970.9
4,20230713,CAD,969.53,989.12,979.33,979,979,979.33
...,...,...,...,...,...,...,...,...
110,20230719,SAR,332.9,339.63,336.27,336,336,336.27
111,20230719,SEK,122.33,124.8,123.57,123,123,123.57
112,20230719,SGD,944.34,963.41,953.88,953,953,953.88
113,20230719,THB,36.66,37.41,37.04,37,37,37.04


In [22]:
# 특정 통화 환율 확인
df_fx_all_save[df_fx_all_save['cur_code'].str.contains('USD')]

,date,cur_code,ttb,tts,deal_bas_r,bkpr,kftc_bkpr,kftc_deal_bas_r
22,20230713,USD,1278.48,1304.31,1291.4,1291,1291,1291.4
45,20230714,USD,1263.43,1288.96,1276.2,1276,1276,1276.2
68,20230717,USD,1253.14,1278.45,1265.8,1265,1265,1265.8
91,20230718,USD,1253.43,1278.76,1266.1,1266,1266,1266.1
114,20230719,USD,1248.98,1274.21,1261.6,1261,1261,1261.6


### 4. 테이블에 데이터 저장 (mdw.weather)

In [28]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_fx_all_save.itertuples():
    sql = "insert into fx_eximbank (date, cur_code, ttb, tts, deal_bas_r, bkpr, kftc_bkpr, kftc_deal_bas_r) \
                           values (%s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8]))
conn.commit()
conn.close()